
"""
# Многоклассовая классификация текстов с пересекающимися классами

**Тематики**: спорт, юмор, реклама, соцсети, политика, личная жизнь

## Этапы:
1. Загрузка и предобработка данных
2. Обучение модели (TF-IDF + LogisticRegression)
3. Оценка качества
4. Сохранение модели

*Примечание: Веб-интерфейс и CLI будут реализованы позже, вероятно, на базе Flask/FastAPI + Vue.js*

In [1]:

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, hamming_loss
import joblib

In [3]:
# %%
# Генерация синтетических данных (далее заменим на свой датасет)
data = {
    'text': [
        'футбольный матч закончился победой нашей сборной',
        'смешной мем про котов в интернете',
        'акция на новый смартфон только сегодня',
        'публикация в инстаграм набрала много лайков',
        'выборы президента пройдут в следующем месяце',
        'встречаемся с друзьями в кафе вечером',
        'чемпионат по баскетболу перенесли на следующую неделю',
        'анекдот про врача и пациента',
        'скидки на все товары в гипермаркете',
        'новый тренд в тиктоке',
        'парламент принял новый закон',
        'семейный ужин в кругу близких'
    ],
    'sport': [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    'humor': [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    'ad': [0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    'social': [0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0],
    'politics': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0],
    'personal': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
}

df = pd.DataFrame(data)
texts = df['text']
labels = df.drop('text', axis=1)


In [4]:

# Разделение данных на train/test
X_train, X_test, y_train, y_test = train_test_split(
    texts, labels, test_size=0.3, random_state=42
)

In [6]:
# Создание и обучение пайплайна (исправленная версия)
model = Pipeline([
    ('tfidf', TfidfVectorizer(
        stop_words=['в', 'на', 'с', 'по', 'про'],  # русские стоп-слова
        ngram_range=(1, 2)
    )),  # Закрываем TfidfVectorizer и его tuple
    ('clf', MultiOutputClassifier(
        LogisticRegression(solver='sag', max_iter=100),
        n_jobs=-1
    ))  # Закрываем MultiOutputClassifier и его tuple
])  # Закрываем Pipeline


In [7]:
# Создание и обучение пайплайна
model = Pipeline([
    ('tfidf', TfidfVectorizer(
        stop_words=['в', 'на', 'с', 'по', 'про'],  # русские стоп-слова
        ngram_range=(1, 2)
    )),
    ('clf', MultiOutputClassifier(
        LogisticRegression(solver='sag', max_iter=100),
        n_jobs=-1
    ))
])

model.fit(X_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(ngram_range=(1, 2),
                                 stop_words=['в', 'на', 'с', 'по', 'про'])),
                ('clf',
                 MultiOutputClassifier(estimator=LogisticRegression(solver='sag'),
                                       n_jobs=-1))])

In [8]:
# Предсказания и оценка качества
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)

print("Classification Report:")
for idx, class_name in enumerate(labels.columns):
    print(f"\nClass {class_name}:")
    print(classification_report(y_test.iloc[:, idx], y_pred[:, idx]))

print("\nHamming Loss:", hamming_loss(y_test, y_pred))

# %%

Classification Report:

Class sport:
              precision    recall  f1-score   support

           0       0.75      1.00      0.86         3
           1       0.00      0.00      0.00         1

    accuracy                           0.75         4
   macro avg       0.38      0.50      0.43         4
weighted avg       0.56      0.75      0.64         4


Class humor:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         4

    accuracy                           1.00         4
   macro avg       1.00      1.00      1.00         4
weighted avg       1.00      1.00      1.00         4


Class ad:
              precision    recall  f1-score   support

           0       0.75      1.00      0.86         3
           1       0.00      0.00      0.00         1

    accuracy                           0.75         4
   macro avg       0.38      0.50      0.43         4
weighted avg       0.56      0.75      0.64         4


Class soci

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

In [9]:
# Сохранение модели
joblib.dump(model, 'multilabel_text_classifier.joblib')

['multilabel_text_classifier.joblib']

In [17]:
# 1. Установим необходимые библиотеки
!pip install -q transformers datasets
!pip install -q scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cud

In [19]:
# 2. Импортируем библиотеки

!pip install -q rarfile

import os
import rarfile
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score


In [20]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

In [21]:
# 3. Распакуем архив dataset.rar
rar_path = '/content/sample_data/dataset.rar'
extract_path = '/content/sample_data/dataset/'

In [22]:
# Убедимся что rarfile работает
rarfile.UNRAR_TOOL = "/usr/bin/unrar"
if not os.path.exists(extract_path):
    with rarfile.RarFile(rar_path) as rf:
        rf.extractall(extract_path)

In [24]:
# Загрузка всех CSV файлов
# Получаем список файлов
csv_files = [os.path.join(extract_path, file) for file in os.listdir(extract_path) if file.endswith('.csv')]

In [27]:
# Читаем все CSV файлы
dataframes = []
for file in csv_files:
    df = pd.read_csv(file, encoding='utf-8', header=None)  # у тебя файлы без заголовков
    dataframes.append(df)

In [30]:
# 5. Предобработка текста: удаление эмодзи
import re

# Функция для удаления эмодзи
def remove_emojis(text):
    emoji_pattern = re.compile("[\U00010000-\U0010ffff]", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)


In [31]:
# Применяем функцию ко всем отзывам
df[0] = df[0].apply(remove_emojis)

In [32]:
# 6. Создаем метки для классификации
# Полагаем, что во втором столбце находятся метки (например, 0 или 1)
df['label'] = df[1]

In [33]:
# 7. Разделим данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(df[0], df['label'], test_size=0.2, random_state=42)

In [34]:
# 8. Используем TfidfVectorizer для преобразования текста в числовые признаки
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [36]:
# 5. Проверим наличие NaN значений в исходных данных
print(f'Проверка на NaN в данных: {df.isnull().sum()}')

Проверка на NaN в данных: 0          0
1        432
2        662
label    432
dtype: int64


In [37]:
# Удаляем строки с NaN в столбцах 1, 2 и label
df = df.dropna(subset=[1, 2, 'label'])


In [38]:
# 6. Повторно разделим данные на обучающую и тестовую выборки после обработки NaN
X_train, X_test, y_train, y_test = train_test_split(df[0], df['label'], test_size=0.2, random_state=42)

In [39]:
# 7. Применим TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [40]:
# 9. Обучение классификатора (например, Logistic Regression)
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_tfidf, y_train)

LogisticRegression(max_iter=1000)

In [41]:
# 10. Оценка модели на тестовой выборке
y_pred = clf.predict(X_test_tfidf)